In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from collections import deque
import heapq
from itertools import count

plt.close('all')

ROWS = 10
COLS = 10
DELAY = 0.1

EMPTY = 0
WALL = -1
VISITED = 1
PATH = 2
FRONTIER = 3

def setup_grid():
    grid = np.zeros((ROWS, COLS), dtype=int)
    
    for r in range(2, 5):
        grid[r, 4] = WALL

    for c in range(2, 5):
        grid[2, c] = WALL

    for r in range(5, 8):
        grid[r, 6] = WALL
    
    return grid

fig = None
ax = None

def draw(grid, title="Algorithm"):
    global fig, ax
    
    if fig is not None:
        plt.clf()
    else:
        fig = plt.figure(figsize=(8, 8))
    
    ax = plt.gca()
    
    color_grid = np.ones((ROWS, COLS, 3))
    
    for r in range(ROWS):
        for c in range(COLS):
            val = grid[r, c]
            
            if val == WALL:
                color_grid[r, c] = [1.0, 0.7, 0.7]
            elif (r, c) == start_pos:
                color_grid[r, c] = [1.0, 0.8, 0.95]
            elif (r, c) == target_pos:
                color_grid[r, c] = [1.0, 0.8, 0.95]
            elif val == VISITED:
                color_grid[r, c] = [0.7, 1.0, 1.0]
            elif val == FRONTIER:
                color_grid[r, c] = [0.8, 1.0, 0.85]
            elif val == PATH:
                color_grid[r, c] = [0.9, 0.7, 0.9]
            else:
                color_grid[r, c] = [1.0, 1.0, 1.0]
    
    ax.imshow(color_grid)
    
    for r in range(ROWS):
        for c in range(COLS):
            val = grid[r, c]
            
            if (r, c) == start_pos:
                text = "S"
                color = "darkgreen"
                fontweight = "bold"
            elif (r, c) == target_pos:
                text = "G"
                color = "darkgreen"
                fontweight = "bold"
            elif val == WALL:
                text = "-1"
                color = "darkred"
                fontweight = "bold"
            else:
                text = str(val) if val >= 0 else "-1"
                color = "black"
                fontweight = "normal"
            
            ax.text(c, r, text, ha='center', va='center', 
                   fontsize=10, color=color, fontweight=fontweight)
    
    for i in range(ROWS + 1):
        ax.axhline(i - 0.5, color='gray', linewidth=1)
    for j in range(COLS + 1):
        ax.axvline(j - 0.5, color='gray', linewidth=1)
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xticks(range(COLS))
    ax.set_yticks(range(ROWS))
    ax.set_xlim(-0.5, COLS - 0.5)
    ax.set_ylim(ROWS - 0.5, -0.5)
    
    plt.tight_layout()
    plt.pause(DELAY)

def get_neighbors(r, c, grid, randomize=False):
    directions = [
        (-1, 0),   
        (0, 1),    
        (1, 0),     
        (1, 1),    
        (0, -1),   
        (-1, -1)   
    ]
    
    if randomize:
        random.shuffle(directions)
    
    result = []
    for dr, dc in directions:
        nr, nc = r + dr, c + dc
        if 0 <= nr < ROWS and 0 <= nc < COLS:
            if grid[nr, nc] != WALL:
                result.append((nr, nc))
    return result

def bfs(grid):
    parent = {}
    visited = {start_pos}
    queue = deque([start_pos])
    nodes_explored = 0
    
    start_time = time.time()
    print("Running BFS")
    
    while queue:
        r, c = queue.popleft()
        nodes_explored += 1
        
        if (r, c) == target_pos:
            elapsed_time = time.time() - start_time
            print("Path found!")
            print(f"Nodes explored: {nodes_explored}")
            print(f"Time: {elapsed_time:.4f}s")
            
            path = []
            current = target_pos
            while current in parent:
                path.append(current)
                current = parent[current]
            
            for r, c in path:
                if (r, c) != start_pos and (r, c) != target_pos:
                    grid[r, c] = PATH
            
            print(f"Path length: {len(path)}\n")
            draw(grid, "BFS - Path Found")
            return
        
        if (r, c) != start_pos:
            grid[r, c] = VISITED
        
        for nr, nc in get_neighbors(r, c, grid, randomize=True):
            if (nr, nc) not in visited:
                if grid[nr, nc] != WALL:
                    visited.add((nr, nc))
                    parent[(nr, nc)] = (r, c)
                    grid[nr, nc] = FRONTIER
                    queue.append((nr, nc))
        
        draw(grid, "BFS")
    
    print("No path found\n")

def dfs(grid):
    parent = {}
    visited = {start_pos}
    stack = [start_pos]
    nodes_explored = 0
    
    start_time = time.time()
    print("Running DFS")
    
    while stack:
        r, c = stack.pop()
        nodes_explored += 1
        
        if (r, c) == target_pos:
            elapsed_time = time.time() - start_time
            print("Path found!")
            print(f"Nodes explored: {nodes_explored}")
            print(f"Time: {elapsed_time:.4f}s")
            
            path = []
            current = target_pos
            while current in parent:
                path.append(current)
                current = parent[current]
            
            for r, c in path:
                if (r, c) != start_pos and (r, c) != target_pos:
                    grid[r, c] = PATH
            
            print(f"Path length: {len(path)}\n")
            draw(grid, "DFS - Path Found")
            return
        
        if (r, c) != start_pos:
            grid[r, c] = VISITED
        
        neighbors = get_neighbors(r, c, grid, randomize=True)
        for nr, nc in neighbors:
            if (nr, nc) not in visited:
                if grid[nr, nc] != WALL:
                    visited.add((nr, nc))
                    parent[(nr, nc)] = (r, c)
                    grid[nr, nc] = FRONTIER
                    stack.append((nr, nc))
        
        draw(grid, "DFS")
    
    print("No path found\n")

def ucs(grid):
    parent = {}
    cost = {start_pos: 0}
    visited = set()
    counter = count()
    heap = [(0, next(counter), start_pos)]
    nodes_explored = 0
    
    start_time = time.time()
    print("Running UCS")
    
    while heap:
        curr_cost, _, (r, c) = heapq.heappop(heap)
        
        if (r, c) in visited:
            continue
        
        visited.add((r, c))
        nodes_explored += 1
        
        if (r, c) == target_pos:
            elapsed_time = time.time() - start_time
            print("Path found!")
            print(f"Nodes explored: {nodes_explored}")
            print(f"Time: {elapsed_time:.4f}s")
            
            path = []
            current = target_pos
            while current in parent:
                path.append(current)
                current = parent[current]
            
            for r, c in path:
                if (r, c) != start_pos and (r, c) != target_pos:
                    grid[r, c] = PATH
            
            print(f"Path length: {len(path)}\n")
            draw(grid, "UCS - Path Found")
            return
        
        if (r, c) != start_pos:
            grid[r, c] = VISITED
        
        for nr, nc in get_neighbors(r, c, grid, randomize=True):
            if (nr, nc) not in visited:
                if grid[nr, nc] != WALL:
                    new_cost = curr_cost + 1
                    if (nr, nc) not in cost or new_cost < cost[(nr, nc)]:
                        cost[(nr, nc)] = new_cost
                        parent[(nr, nc)] = (r, c)
                        grid[nr, nc] = FRONTIER
                        heapq.heappush(heap, (new_cost, next(counter), (nr, nc)))
        
        draw(grid, "UCS")
    
    print("No path found\n")

def dls(grid, limit):
    parent = {}
    visited = set()
    found = [False]
    nodes_explored = [0]
    
    start_time = time.time()
    print(f"Running DLS (limit={limit})")
    
    def dls_rec(node, depth):
        if found[0]:
            return
        
        nodes_explored[0] += 1
        r, c = node
        visited.add(node)
        
        if node == target_pos:
            found[0] = True
            return
        
        if depth == 0:
            return
        
        if node != start_pos:
            grid[r, c] = VISITED
        
        for nr, nc in get_neighbors(r, c, grid, randomize=True):
            if (nr, nc) not in visited and grid[nr, nc] != WALL:
                grid[nr, nc] = FRONTIER
                parent[(nr, nc)] = node
                draw(grid, f"DLS (limit={limit})")
                dls_rec((nr, nc), depth - 1)
    
    dls_rec(start_pos, limit)
    
    elapsed_time = time.time() - start_time
    
    if found[0]:
        print("Path found!")
        print(f"Nodes explored: {nodes_explored[0]}")
        print(f"Time: {elapsed_time:.4f}s")
        
        path = []
        current = target_pos
        while current in parent:
            path.append(current)
            current = parent[current]
        
        for r, c in path:
            if (r, c) != start_pos and (r, c) != target_pos:
                grid[r, c] = PATH
        
        print(f"Path length: {len(path)}\n")
        draw(grid, f"DLS - Path Found (limit={limit})")
    else:
        print("No path found\n")

def iddfs(grid):
    nodes_explored = [0]
    start_time = time.time()
    print("Running IDDFS")
    
    for depth in range(15):
        parent = {}
        visited = set()
        found = [False]
        
        def iddfs_rec(node, curr_depth):
            if found[0]:
                return
            
            nodes_explored[0] += 1
            r, c = node
            visited.add(node)
            
            if node == target_pos:
                found[0] = True
                return
            
            if curr_depth == 0:
                return
            
            if node != start_pos:
                grid[r, c] = VISITED
            
            for nr, nc in get_neighbors(r, c, grid, randomize=True):
                if (nr, nc) not in visited and grid[nr, nc] != WALL:
                    grid[nr, nc] = FRONTIER
                    parent[(nr, nc)] = node
                    if depth < 3:
                        draw(grid, f"IDDFS (depth={depth})")
                    iddfs_rec((nr, nc), curr_depth - 1)
        
        iddfs_rec(start_pos, depth)
        
        if found[0]:
            elapsed_time = time.time() - start_time
            print("Path found!")
            print(f"Nodes explored: {nodes_explored[0]}")
            print(f"Time: {elapsed_time:.4f}s")
            
            path = []
            current = target_pos
            while current in parent:
                path.append(current)
                current = parent[current]
            
            for r, c in path:
                if (r, c) != start_pos and (r, c) != target_pos:
                    grid[r, c] = PATH
            
            print(f"Path length: {len(path)}\n")
            draw(grid, "IDDFS - Path Found")
            return
    
    elapsed_time = time.time() - start_time
    print("No path found\n")

def bidirectional(grid):
    parent_f = {}
    parent_b = {}
    visited_f = {start_pos}
    visited_b = {target_pos}
    queue_f = deque([start_pos])
    queue_b = deque([target_pos])
    nodes_explored = 0
    
    start_time = time.time()
    print("Running Bidirectional Search")
    
    while queue_f and queue_b:
        if queue_f:
            r, c = queue_f.popleft()
            nodes_explored += 1
            if (r, c) != start_pos:
                grid[r, c] = VISITED
            
            for nr, nc in get_neighbors(r, c, grid, randomize=True):
                if (nr, nc) not in visited_f and grid[nr, nc] != WALL:
                    visited_f.add((nr, nc))
                    parent_f[(nr, nc)] = (r, c)
                    grid[nr, nc] = FRONTIER
                    queue_f.append((nr, nc))
                    
                    if (nr, nc) in visited_b:
                        elapsed_time = time.time() - start_time
                        print("Path found!")
                        print(f"Nodes explored: {nodes_explored}")
                        print(f"Time: {elapsed_time:.4f}s")
                        
                        path = []
                        current = (nr, nc)
                        while current in parent_f:
                            path.append(current)
                            current = parent_f[current]
                        path.append(start_pos)
                        path.reverse()
                        
                        path2 = []
                        current = (nr, nc)
                        while current in parent_b:
                            current = parent_b[current]
                            if current != target_pos:
                                path2.append(current)
                        path.extend(reversed(path2))
                        path.append(target_pos)
                        
                        for r, c in path:
                            if (r, c) != start_pos and (r, c) != target_pos:
                                grid[r, c] = PATH
                        
                        print(f"Path length: {len(path)}\n")
                        draw(grid, "Bidirectional - Path Found")
                        return
        
        if queue_b:
            r, c = queue_b.popleft()
            nodes_explored += 1
            if (r, c) != target_pos:
                grid[r, c] = VISITED
            
            for nr, nc in get_neighbors(r, c, grid, randomize=True):
                if (nr, nc) not in visited_b and grid[nr, nc] != WALL:
                    visited_b.add((nr, nc))
                    parent_b[(nr, nc)] = (r, c)
                    grid[nr, nc] = FRONTIER
                    queue_b.append((nr, nc))
                    
                    if (nr, nc) in visited_f:
                        elapsed_time = time.time() - start_time
                        print("Path found!")
                        print(f"Nodes explored: {nodes_explored}")
                        print(f"Time: {elapsed_time:.4f}s")
                        
                        path = []
                        current = (nr, nc)
                        while current in parent_f:
                            path.append(current)
                            current = parent_f[current]
                        path.append(start_pos)
                        path.reverse()
                        
                        path2 = []
                        current = (nr, nc)
                        while current in parent_b:
                            current = parent_b[current]
                            if current != target_pos:
                                path2.append(current)
                        path.extend(reversed(path2))
                        path.append(target_pos)
                        
                        for r, c in path:
                            if (r, c) != start_pos and (r, c) != target_pos:
                                grid[r, c] = PATH
                        
                        print(f"Path length: {len(path)}\n")
                        draw(grid, "Bidirectional - Path Found")
                        return
        
        draw(grid, "Bidirectional")
    
    print("No path found\n")

start_pos = (6, 1)
target_pos = (6, 8)

def main():
    while True:
        print("\nSelect Algorithm:")
        print("1 = BFS")
        print("2 = DFS")
        print("3 = UCS")
        print("4 = DLS")
        print("5 = IDDFS")
        print("6 = Bidirectional")
        print("Q = Quit")
        
        choice = input("\nEnter choice: ").strip().upper()
        
        if choice == 'Q':
            break
        
        grid = setup_grid()
        
        if choice == '1':
            bfs(grid)
        elif choice == '2':
            dfs(grid)
        elif choice == '3':
            ucs(grid)
        elif choice == '4':
            dls(grid, 15)
        elif choice == '5':
            iddfs(grid)
        elif choice == '6':
            bidirectional(grid)

if __name__ == "__main__":
    main()